In [1]:
import warnings
import numpy as np
from six.moves import reload_module as reload
from sklearn.utils.extmath import fast_logdet
from sklearn.utils import check_random_state
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.covariance import empirical_covariance
from functools import partial

from network_inference.prox import prox_logdet, soft_thresholding_od
import network_inference.datasets; reload(network_inference.datasets)
from network_inference.datasets import is_pos_def, is_pos_semi_def
from network_inference.utils import _scalar_product, update_rho, convergence
from sklearn.utils.extmath import squared_norm
from network_inference.utils import l1_od_norm
from scipy import linalg

In [15]:
np.random.seed(0)
random_state=0
A = make_sparse_spd_matrix(dim=105, alpha=0.7, random_state=random_state)

T_true = A[5:,5:]
K_true = A[100:,5:,]
H_true = A[0:5,0:5]
per_cov = K_true*0.3
T_obs = T_true - per_cov.T.dot(np.linalg.inv(H_true)).dot(per_cov)

samples = np.random.multivariate_normal(np.zeros(100), np.linalg.inv(T_obs), 500)

In [3]:
def objective_H(H, R=None, T=None, K=None, U= None,_rho=1, _mu=1):
    if not is_pos_def(H):
        return np.inf
    return 0.5 * _rho * squared_norm(R - T + U + np.linalg.multi_dot((K.T, linalg.pinvh(H), K))) \
            + _mu * l1_od_norm(H)

In [4]:
def _choose_lambda(lamda, R, T, K, H, U,  _rho, _mu, prox, grad, gamma, delta=1e-4, eps=0.9, max_iter=500):
    """Choose lambda for backtracking.

    References
    ----------
    Salzo S. (2017). https://doi.org/10.1137/16M1073741

    """
    partial_f = partial(objective_H, R=R, T=T, K=K, U=U, _rho=_rho, _mu=_mu)
    fx = partial_f(H)

    y_minus_x = prox - H
    tolerance = _scalar_product(y_minus_x, grad)
    tolerance += delta / gamma * _scalar_product(y_minus_x, y_minus_x)
    #print("Tolerance:", tolerance)
    for i in range(max_iter):
        # line-search
        x1 = H + lamda * y_minus_x

        loss_diff = partial_f(x1) - fx
        #print("Loss diff:", loss_diff)
        if loss_diff <= lamda * tolerance:
              break
        lamda *= eps
    else:
        print("not found lambda")
    return lamda, i + 1

In [5]:
def _choose_gamma(gamma, H, R, T, K, U, _rho, _mu, _lambda, grad,
                 eps=0.9, max_iter=500):
    """Choose gamma for backtracking.

    References
    ----------
    Salzo S. (2017). https://doi.org/10.1137/16M1073741

    """
    partial_f = partial(objective_H, R=R, T=T, K=K, U=U, _rho=_rho, _mu=_mu)
    fx = partial_f(H)
    for i in range(max_iter):
        prox = soft_thresholding_od(H - gamma * grad, _mu * gamma)
        if is_pos_def(prox):
            break
        gamma *= eps
    else:
        print("not found gamma")
    return gamma, prox

In [6]:
def _upgrade_H(H, R, T, K, U, _rho, _mu, verbose=0, random_state=None):
    # H = make_sparse_spd_matrix(dim=K.shape[0], alpha=0.5, random_state=random_state)
    _lambda = 1
    gamma = 1
    obj = 1e+10
    for iter_ in range(2000):
        H_old = H.copy()
        Hinv = linalg.pinvh(H)
        gradient =  -_rho* K.dot(R - T + U + np.linalg.multi_dot((K.T, Hinv, K))).dot(K.T).dot(Hinv).dot(Hinv)
        gamma, _ = _choose_gamma(gamma, H, R, T, K, U, _rho,_mu, _lambda, gradient)
        Y = soft_thresholding_od(H - gamma * gradient, gamma * _mu)
        _lambda,_ = _choose_lambda(_lambda, R, T, K, H, U,_rho, _mu, Y, gradient, gamma, max_iter=1000)
        H = H + _lambda * (Y - H)
        assert is_pos_def(H)
        obj_old = obj
        obj = objective_H(H, R, T, K, U,_rho=_rho, _mu=_mu)
        obj_diff = obj_old - obj
        iter_diff =np.linalg.norm(H - H_old) 
        if verbose:
            print("Iter: %d, obj: %.5f, iter_diff: %.5f, obj_diff:%.10f"%(iter_, obj, iter_diff, obj_diff))
        if(obj_diff<1e-4): 
            break
    else:
        print("Did not converge")
    return H

In [7]:
R = np.random.rand(10,10)
T = make_sparse_spd_matrix(dim=10, alpha=0.5)*20
U = np.zeros((10,10))
K = per_cov*0.1
#print(R.shape, T.shape, U.shape, K.shape)

H = make_sparse_spd_matrix(dim=K.shape[0], alpha=0.5)*10
#print(H)
#print(R - T + U + np.linalg.multi_dot((K.T, H, K)))
R = T - K.T.dot(np.linalg.pinv(H)).dot(K)
print(is_pos_semi_def(T-R))
#print(R - T + K.T.dot(np.linalg.pinv(H)).dot(K) )
print(is_pos_def(R))
print(is_pos_semi_def(K.T.dot(np.linalg.pinv(H)).dot(K)))
H_found1 = _upgrade_H(make_sparse_spd_matrix(dim=K.shape[0], alpha=0.2),
                     R, T, K, U, 0.5,0.003, 1, random_state=0)
#H_found[np.where(np.abs(H_found)<1e-5)] = 0

H_found2 = _upgrade_H(make_sparse_spd_matrix(dim=K.shape[0], alpha=0.2),
                     R, T, K, U, 0.5,0.003, 1, random_state=0)

H_found3 = _upgrade_H(make_sparse_spd_matrix(dim=K.shape[0], alpha=0.2),
                     R, T, K, U, 0.5,0.003, 1, random_state=0)

H_found1 , H_found2 , H_found3, H_true

False
True
True
Iter: 0, obj: 0.01669, iter_diff: 0.01208, obj_diff:9999999999.9833126068
Iter: 1, obj: 0.01654, iter_diff: 0.01207, obj_diff:0.0001455659
Iter: 2, obj: 0.01640, iter_diff: 0.01207, obj_diff:0.0001454692
Iter: 3, obj: 0.01625, iter_diff: 0.01207, obj_diff:0.0001453807
Iter: 4, obj: 0.01611, iter_diff: 0.01206, obj_diff:0.0001452993
Iter: 5, obj: 0.01596, iter_diff: 0.01206, obj_diff:0.0001452244
Iter: 6, obj: 0.01581, iter_diff: 0.01205, obj_diff:0.0001451553
Iter: 7, obj: 0.01567, iter_diff: 0.01205, obj_diff:0.0001450914
Iter: 8, obj: 0.01552, iter_diff: 0.01205, obj_diff:0.0001450323
Iter: 9, obj: 0.01538, iter_diff: 0.01205, obj_diff:0.0001449775
Iter: 10, obj: 0.01523, iter_diff: 0.01204, obj_diff:0.0001449266
Iter: 11, obj: 0.01509, iter_diff: 0.01204, obj_diff:0.0001448792
Iter: 12, obj: 0.01495, iter_diff: 0.01204, obj_diff:0.0001448350
Iter: 13, obj: 0.01480, iter_diff: 0.01204, obj_diff:0.0001447938
Iter: 14, obj: 0.01466, iter_diff: 0.01203, obj_diff:0.000144

(array([[ 1.55023257,  0.        ,  0.        , -0.6212299 , -0.02983354],
        [ 0.        ,  2.03659726,  0.        , -0.03452956, -0.73565838],
        [ 0.        ,  0.        ,  1.46861319,  0.        , -0.57980673],
        [-0.62254823, -0.03459194,  0.        ,  1.00105197,  0.        ],
        [-0.03035457, -0.73589221, -0.57917674,  0.        ,  1.00077523]]),
 array([[ 1.21068059,  0.03529319, -0.38400949, -0.21352122, -0.09562236],
        [ 0.03527721,  1.16490651,  0.        ,  0.        , -0.40004162],
        [-0.38403334,  0.        ,  0.99999794,  0.        ,  0.        ],
        [-0.21352601,  0.        ,  0.        ,  1.00002254,  0.        ],
        [-0.09564479, -0.4000552 ,  0.        ,  0.        ,  1.0000066 ]]),
 array([[ 1.97635268,  0.        , -0.42354303,  0.        , -0.13501885],
        [ 0.        ,  1.53267468, -0.15403615, -0.22384937,  0.        ],
        [-0.42523562, -0.15271166,  1.25778998, -0.29485871,  0.        ],
        [ 0.        ,

In [8]:
def objective(emp_cov, K, R, T, H, U,mu, eta, rho):
    if not is_pos_def(H):
        return np.inf
    res = - fast_logdet(R) + np.sum(R * emp_cov)
    #print(res)
    res += rho / 2. * squared_norm(R - T + U + np.linalg.multi_dot((K.T, linalg.pinvh(H), K)))
    #print(squared_norm(R + U + np.linalg.multi_dot((K.T, linalg.pinvh(H), K))))
    res += mu * l1_od_norm(H)
    #print(l1_od_norm(H))
    res += eta * l1_od_norm(T)
    #print(l1_od_norm(T))
    return res

In [9]:
def fixed_interlinks_graphical_lasso(X, K, mu=0.01, eta=0.01, rho=1., 
        tol=1e-3, rtol=1e-5, max_iter=100, verbose=False, return_n_iter=True,
        return_history=False, compute_objective=False, compute_emp_cov=False,
        random_state=None, update_rho=False):
    
    random_state = check_random_state(random_state)
    if compute_emp_cov:
        n = X.shape[0] 
        emp_cov = empirical_covariance(X, assume_centered=False)
    else:
        emp_cov = X

    H = make_sparse_spd_matrix(K.shape[0], alpha=0.2, random_state=random_state)
    print(H)
    #H = np.eye(K.shape[0])
    T = linalg.pinvh(emp_cov.copy() + 1e-5*np.eye(emp_cov.shape[0]))*10
    #T = (T + T.T) / 2.
    R = np.zeros((K.shape[1], K.shape[1]))
    U = np.zeros((K.shape[1], K.shape[1]))
    
    checks = []
    for iteration_ in range(max_iter):
        R_old = R.copy()
        
        # R update
        Hinv = linalg.pinvh(H)
        M = T - U - K.T.dot(Hinv).dot(K)
        M = (M + M.T)/2
        R = prox_logdet(emp_cov - rho * M,  1/rho)
        assert is_pos_def(R), "R non positive deinife iter %d"%iteration_
        
        # T update
        M = R + U + K.T.dot(Hinv).dot(K)
        T = soft_thresholding_od(M, eta / rho)
        assert is_pos_def(T, tol=1e-8), "Theta non positive definite iter %d"%iteration_
       
        # H update
        KHK = np.linalg.multi_dot((K.T, linalg.pinvh(H), K))
        H = _upgrade_H(H, R, T, K, U, rho, mu, verbose=0)
        
        # U update
        KHK = np.linalg.multi_dot((K.T, linalg.pinvh(H), K))
        U += R - T + KHK
        assert is_pos_semi_def(KHK), "KHK non positive semi-definite iter %d"%iteration_
        
        # diagnostics, reporting, termination checks
        obj = objective(emp_cov, K, R, T, H, U, mu, eta, rho) \
            if compute_objective else np.nan
        rnorm = np.linalg.norm(R - T + KHK)
        snorm = rho * np.linalg.norm(R - R_old)
        check = convergence(
            obj=obj, rnorm=rnorm, snorm=snorm,
            e_pri=(np.sqrt(R.size) * tol + rtol *
                   max(np.linalg.norm(R),
                       np.linalg.norm(T - KHK))),
            e_dual=(np.sqrt(R.size) * tol + rtol * rho *
                    np.linalg.norm(U))
        )
        
        if verbose:
            print("obj: %.4f, rnorm: %.4f, snorm: %.4f,"
                  "eps_pri: %.4f, eps_dual: %.4f" % check)

        checks.append(check)
        if check.rnorm <= check.e_pri and check.snorm <= check.e_dual:
            break
        if update_rho:
            rho_new = update_rho(rho, rnorm, snorm, iteration=iteration_)
            # scaled dual variables should be also rescaled
            U *= rho / rho_new
            rho = rho_new
    else:
        warnings.warn("Objective did not converge.")

    return_list = [R, T, H, emp_cov]
    if return_n_iter:
        return_list.append(iteration_)
    if return_history:
        return_list.append(checks)
    return return_list


In [17]:
res = fixed_interlinks_graphical_lasso(samples, per_cov*0.00001, mu=0.001, eta=0.5, rho=1., 
        verbose=1, compute_objective=1, compute_emp_cov=1,
        random_state=0)

[[ 1.09712229  0.05691441 -0.31164449  0.         -0.40452597]
 [ 0.05691441  2.26141742 -0.65810496 -0.22766968 -0.15504919]
 [-0.31164449 -0.65810496  1.          0.         -0.5941084 ]
 [ 0.         -0.22766968  0.          1.41661376 -0.64545624]
 [-0.40452597 -0.15504919 -0.5941084  -0.64545624  1.70068326]]


AssertionError: Theta non positive definite iter 0

In [11]:
LL = res[2].copy()
LL[np.where(np.abs(LL)<1e-4)] = 0
LL

array([[ 1.10706338,  0.        , -0.21533349,  0.        , -0.3109012 ],
       [ 0.        ,  2.26273026, -0.57124563, -0.14305028, -0.06887208],
       [-0.23184238, -0.57604456,  0.99527511,  0.        , -0.51388937],
       [ 0.        , -0.14392242,  0.        ,  1.41727544, -0.56136767],
       [-0.31695746, -0.06886739, -0.50446247, -0.55911651,  1.70489515]])

In [12]:
H_true

array([[ 1.22170946,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  2.07826367,  0.        , -0.15206798,  0.74181422],
       [ 0.        ,  0.        ,  1.        ,  0.        , -0.85945506],
       [ 0.        , -0.15206798,  0.        ,  2.22706349,  0.        ],
       [ 0.        ,  0.74181422, -0.85945506,  0.        ,  2.4664463 ]])

In [13]:
res[1]

array([[ 0.63708101, -0.        , -0.        , -0.        , -0.        ,
        -0.12095312,  0.        , -0.02421178,  0.        ,  0.        ],
       [-0.        ,  1.02897462, -0.        , -0.07776693, -0.        ,
        -0.        , -0.        , -0.        , -0.        , -0.13195498],
       [-0.        , -0.        ,  1.54140573, -0.        ,  0.        ,
        -0.        , -0.        , -0.        , -0.        , -0.        ],
       [-0.        , -0.07776693, -0.        ,  0.7399214 ,  0.        ,
        -0.        , -0.        , -0.        , -0.        , -0.        ],
       [-0.        , -0.        ,  0.        ,  0.        ,  0.99582247,
        -0.        , -0.        , -0.        , -0.        , -0.        ],
       [-0.12095312, -0.        , -0.        , -0.        , -0.        ,
         1.22090485,  0.        , -0.05392683,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.        , -0.        , -0.        ,
         0.        ,  0.58094743, -0.1015059 

In [14]:
T_true

array([[ 1.        ,  0.        , -0.16673795,  0.        ,  0.        ,
        -0.88199611,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  2.04044801,  0.        , -0.63593324,  0.        ,
         0.        , -0.32538408,  0.        ,  0.27733968, -0.72812233],
       [-0.16673795,  0.        ,  1.32129891,  0.        ,  0.        ,
         0.14706222,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.63593324,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.88199611,  0.        ,  0.14706222,  0.        ,  0.        ,
         2.55713568,  0.44253476, -0.8012042 ,  0.        ,  0.        ],
       [ 0.        , -0.32538408,  0.        ,  0.        ,  0.        ,
         0.44253476,  1.30507622, -0.55233705